In [ ]:
#| default_exp sampler.tree

This data structure is based on the Python package `anytree` (https://github.com/c0fec0de/anytree)  which has been extended by the classes `Constant`, `Variable` and `CountTree`.

In [ ]:
#| include: false
from nbdev.showdoc import *

In [ ]:
#| export

from anytree import NodeMixin, RenderTree, PreOrderIter
import qsample.math as math
import pydot
import dill as pickle

In [ ]:
#| export            
class Node(NodeMixin):
    
    def __init__(self, name, parent=None, children=None):
        self.name = name
        self.parent = parent
        self.id = id(self)
        if children:
            self.children = children
            
class Variable(Node):
    
    def __init__(self, name, parent=None, counts=0, marked=False, **kwargs):
        super().__init__(name, parent)
        self.counts = counts
        self.marked = marked
       
    @property
    def rate(self):
        if self.parent.counts == 0: return 0
        else: return self.counts / self.parent.counts
    
    def __str__(self):
        return f"{self.name} ({self.counts}/{self.parent.counts})"
    
class Constant(Node):
    
    def __init__(self, name, parent=None, cid=None, counts=0, is_deterministic=False, **kwargs):
        super().__init__(name, parent)
        self.cid = cid
        self.counts = counts
        self.is_deterministic = is_deterministic
    
    @property
    def variance(self):
        """Variance of children (Variable) nodes."""
        if self.is_root or self.counts == 0: return 0
        else: return math.Wilson_var(self.children[0].rate, self.counts)
    
    def __str__(self):
        return f"{self.name} ({self.counts}, {self.variance:.2e})"

In [ ]:
#| export 
class CountTree:
    
    def __init__(self, constants=None, min_path_weight=1):
        self.constants = constants
        self.root = None
        self.min_path_weight = min_path_weight
        
    def add(self, name, nodetype=None, parent=None, **kwargs):
        if parent is None:
            if self.root is None:
                self.root = Constant(name, **kwargs)
            return self.root
        
        childmatch = [node for node in parent.children if node.name == name]
        if childmatch:
            return childmatch[0]
        else:
            return nodetype(name, parent=parent, **kwargs)
        
    def detach(self, node):
        node.parent = None
        
    def multiplier(self, node):
        if node.is_root: return 1
        elif isinstance(node, Variable): return node.rate
        elif isinstance(node, Constant): return self.constants[node.cid][node.name]
        else: raise TypeError(f"Node of type {node.__class__} not recognized.")
        
    @property
    def direct_rate(self):
        cnts = self.root.counts
        fail_cnts = sum([n.counts for n in self.root.leaves if n.marked])
        return fail_cnts / cnts
    
    @property
    def direct_variance(self):
        return math.Wilson_var(self.direct_rate, self.root.counts)
        
    @property
    def norm(self):
        acc = 0.0
        for leaf in self.root.leaves:
            if not leaf.is_root:
                prod = 1.0
                for node in leaf.path:
                    prod *= self.multiplier(node)
                acc += prod
        return acc
    
    @property
    def delta(self):
        return 1.0 - self.norm
    
    @property
    def rate(self):
        acc = 0.0
        for leaf in self.root.leaves:
            if leaf.marked:
                prod = 1.0
                for node in leaf.path:
                    prod *= self.multiplier(node)
                acc += prod
        return acc
    
    def twig(self, node):
        twig = 1.0
        for node in node.path:
            twig *= self.multiplier(node)
        return twig
    
    def subtree(self, node, all_leaves=False):
        path_sum = 0.0
                
        for leaf in node.leaves:
            if leaf.marked or all_leaves:
                
                prod = 1.0
                for n in leaf.iter_path_reverse():
                    if n == node:
                        break
                    else:
                        prod *= self.multiplier(n)
                path_sum += prod
        return path_sum
    
    def subtreediff(self, node, all_leaves):
        assert( isinstance(node, Constant) )
        subtrees = []
        for child in node.children:
            subtree = self.subtree(child, all_leaves)
            subtrees.append(subtree)
        if len(subtrees) == 1: 
            return subtrees[0]
        elif len(subtrees) == 2: return subtrees[0] - subtrees[1]
        else: raise Exception(f"Node {node.name} must have <= 2 children.")
            
    def partial_variance(self, node, all_leaves=False):
        assert isinstance(node, Constant)
        subtreediff = self.subtreediff(node, all_leaves)
        return node.variance * subtreediff**2
    
    def is_invariant_path(self, leaf):
        path_weight = sum([sum(n.name) for n in leaf.path[1:] if isinstance(n,Constant)])
        return path_weight < self.min_path_weight
        
    @property
    def variance(self):
        acc = 0.0
        constants = [node for node in self.root.descendants if isinstance(node, Constant)]
        for node in constants:
            
            if node.is_deterministic or (all([c.is_leaf for c in node.children]) and self.is_invariant_path(node)):
                continue # exclude deterministic nodes and invariant paths
                
            twig = self.twig(node)
            if len(node.children) == 1 and node.children[0].is_leaf:
                acc += twig**2 * node.variance
            else:
                acc += twig**2 * self.partial_variance(node)
        return acc

    @property
    def norm_variance(self):
        acc = 0.0
        constants = [node for node in self.root.descendants if isinstance(node, Constant)]
        for node in constants:
            if node.is_deterministic or self.is_invariant_path(node):
                continue
            elif all([n.is_leaf for n in node.children]): 
                continue
            twig = self.twig(node)
            acc += twig**2 * self.partial_variance(node, all_leaves=True)
        return acc
    
    def __str__(self):
        return '\n'.join([f'{pre}{node}' for pre, _, node in RenderTree(self.root)])
    
    def save(self, path):
        import numpy as np
        data = self.to_dict()
        data["constants"] = {k: v.tolist() if isinstance(v, np.ndarray) else v for k,v in self.constants.items()}
        with open(path, 'wb') as fp:
            pickle.dump(data, fp)
    
    def load(self, path):
        with open(path, 'rb') as fp:
            data = pickle.load(fp)
        self.constants = data.pop("constants")
        self.from_dict(data)
    
    def to_dict(self, node=None):
        if node == None: node = self.root
        data = dict( list(self._iter_node_attr(node)) )
        data["node_cls"] = node.__class__.__name__
        children = list( self.to_dict(child) for child in node.children )
        if children:
            data["children"] = children
        return data
    
    def from_dict(self, data):
        self.root = self.__import(data)
        
    @staticmethod
    def __import(data, parent=None):
        assert isinstance(data, dict)
        assert "parent" not in data
        children = data.pop("children", [])
        node_name = data.pop("name")
        node_cls = globals()[ data.pop("node_cls") ]
        node = node_cls(node_name, parent=parent, **data)
        for child in children:
            CountTree.__import(child, parent=node)
        return node
    
    @staticmethod
    def _iter_node_attr(node):
        for k, v in node.__dict__.items():
            if k in ("_NodeMixin__children", "_NodeMixin__parent"):
                continue
            yield k, v
            
    def draw(self, path=None, scale_percent=100):
        
        def gen_node(node):
            if isinstance(node, Variable):
                color = "#ff9999" if node.marked else "white"
                return pydot.Node(hex(node.id), label=str(node.name), style="filled", fillcolor=color, shape="ellipse")
            if isinstance(node, Constant):
                return pydot.Node(hex(node.id), label=str(node.name), style="filled", fillcolor="white", shape="box")
            
        def edgeattrfunc(node, child):
            weight = 0 if self.root.counts == 0 else 10.0 * child.counts / self.root.counts 
            weight = min(weight, 5.0) if weight > 5.0 else max(weight, 0.2)
            return {"penwidth": str(weight)}
        
        G = pydot.Dot(graph_type="digraph")
        
        for node in PreOrderIter(self.root):
            nodeA = gen_node(node)
            G.add_node(nodeA)
            for child in node.children:
                nodeB = gen_node(child)
                edge = pydot.Edge(nodeA,nodeB,**edgeattrfunc(node,child))
                G.add_edge(edge)
                
        if path is not None:
            G.write_png(path)
        else:
            try:
                from IPython.display import Image, display
                return display( Image(G.create_png()) )
            except:
                print('Image can only be drawn in Jupyter notebook.')